In [190]:
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
plt.rcParams["figure.figsize"] = (6,4)
plt.rcParams['figure.dpi'] = 96
sns.set_style("darkgrid")
import glob
import math
import os
import random
import shutil
shutil.rmtree('./outputs/P04/')
os.mkdir('./outputs/P04/')

In [191]:
# Loading images
imgs = []
labels = []
label_imgs = {}

for img in glob.glob("./inputs/P4/*.png"):
    lb = " ".join(list(os.path.splitext(os.path.basename(img))[0].split("_"))).title()
    img = cv2.imread(img)    
    labels.append(lb)
    imgs.append(img)
    label_imgs[lb] = img
    if lb == "Maze":
        lb = "Maze_Empty"
        labels.append(lb)
        imgs.append(img)
        label_imgs[lb] = img
        
    
imgs = np.array(imgs)
labels = np.array(labels)

<ipython-input-191-8e4f79d89402>:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  imgs = np.array(imgs)


In [192]:
GhostsNames = ["Blinky", "Clyde","Inky","Pinky"]
GhostsLegs = [1,1,1,1]
Dirs = ["Left","Right","Up","Down"]
PacM = ["Half","Open","Closed"]
Moves = ["Move1","Move2"]
AteDotsPos = []
FrameID = 0
PacDir = "Left"
Locations = []
States = []
ALLDOTS =[]
PowerLocations = []

In [193]:
def PosWriter(pos_:tuple,map_:np.array,item:np.array):
    pos = list(pos_)
    if item.shape[0]%2==0:
        start_row = pos[0]- (item.shape[0]//2)
        end_row = pos[0]+ (item.shape[0]//2)
    else:
        start_row = pos[0]- (item.shape[0]//2) -1
        end_row = pos[0]+ (item.shape[0]//2) 

    if item.shape[1]%2==0:
        start_col = pos[1]- (item.shape[1]//2)
        end_col = pos[1]+ (item.shape[1]//2)
        
    else:
        start_col = pos[1]- (item.shape[1]//2) -1
        end_col = pos[1]+ (item.shape[1]//2)     

    map_ [start_row:end_row,start_col:end_col,:] = item
    return map_    
    

In [194]:
def init_game(maze_img,item_imgs):
    find_dots = cv2.matchTemplate(maze_img, item_imgs["Dot"], cv2.TM_CCORR_NORMED)
    dots_location_ = np.where(find_dots >= 0.99999)
    dots_location_ = list(zip(*dots_location_))
    global ALLDOTS
    ALLDOTS = dots_location_.copy()
    
    find_power = cv2.matchTemplate(maze_img, item_imgs["Power Pellet"], cv2.TM_CCORR_NORMED)
    power_location_ = np.where(find_power >= 0.9999)
    power_location_ = list(zip(*power_location_))
    
    for p_ in power_location_:
        for i in range(9):
            for j in range(9):
                p = list(p_).copy()
                p[0] += i
                p[1] += j
                ALLDOTS.append(tuple(p))
                PowerLocations.append(tuple(p))
   

    dots_location = random.sample(dots_location_,5)
    
    tmp = maze_img.copy()
    pac = item_imgs["Pacman Closed"]
    tmp = PosWriter(dots_location[0],tmp,pac)
    
    global AteDotsPos
    AteDotsPos.append(dots_location[0])
    
    state = ["Pacman Closed"]
    for index in range(len(GhostsNames)):
        ghost = ""
        move = random.sample(Moves,1)[0]
        GhostsLegs[index] = move
        ghost += " " + move
        ghost += " " + random.sample(Dirs,4)[0]
        state.append(ghost)
        ghost = GhostsNames[index] + ghost
        tmp = PosWriter(dots_location[index+1],tmp,item_imgs[ghost])  
                     
    global Locations
    global States
    Locations = dots_location
    States = state
    
    cv2.imwrite("./outputs/P04/1-init-map.png",tmp)
    
    global FrameID
    FrameID += 1
    return tmp


In [195]:
def GetNewPos(pos,dir_):
    pos = pos.copy()
    if dir_ == "Left":
           pos[1] -= 3

    if dir_ == "Right":
        pos[1] += 3
        
    if dir_ == "Up":
           pos[0] -= 3

    if dir_ == "Down":
        pos[0] += 3
    
    return pos

In [196]:
def Access(pos_,dir_,self_flag=True):
    global ALLDOTS
    for i in range(1,9):
        pos = pos_.copy()
        if dir_ == "Left":
            pos[1] -= i

        if dir_ == "Right":
            pos[1] += i
            
        if dir_ == "Up":
            pos[0] -= i

        if dir_ == "Down":
            pos[0] += i
        
        pos = tuple(pos)  
        
        if  pos in ALLDOTS :
            return True
        
    return False
  
        

In [197]:
def GhostMove(current,pos,_map):    
    global Dirs        
    pos = list(pos)
    flag = False
    dir_ = list(current.split())[2]
    while not Access(pos,dir_):
        paths = Dirs.copy()
        if dir_ == "Left" or dir_ =="Right":  
            del paths[paths.index("Right")]     
            del paths[paths.index("Left")]  

        if dir_ == "Up" or dir_ =="Down":
            del paths[paths.index("Up")]     
            del paths[paths.index("Down")]
              
        dir_ = random.sample(paths,1)[0]
        current = list(current.split())
        del current [2]
        current.insert(2,dir_)        
        current = " ".join(current)
        
    pos = GetNewPos(pos,dir_)
    _map = PosWriter(pos,_map,label_imgs[current])
        
    return _map,pos,current

In [198]:
def IsNotAte(pos,dir_,look):
    global AteDotsPos
    for thresh_look in range(look):
        if dir_ == "Left":
            pos_ = pos.copy()
            pos_[1] -= thresh_look
            pos_ = tuple(pos_)
        if dir_ == "Right":
            pos_ = pos.copy()
            pos_[1] += thresh_look
            pos_ = tuple(pos_)
        if dir_ == "Down":
            pos_ = pos.copy()
            pos_[1] += thresh_look 
            pos_ = tuple(pos_)
        if dir_ == "Up":
            pos_ = pos.copy()
            pos_[1] -= thresh_look
            pos_ = tuple(pos_)

        if pos_ in AteDotsPos :
            return False 
        
    return True
        

In [199]:
def EatPos(pos_,dir_):
    global label_imgs
    for i in range(0,4):
        pos = pos_.copy()
        if dir_ == "Left":
            pos[1] += i
            AteDotsPos.append(tuple(pos))
            pos[0] -= 1
            AteDotsPos.append(tuple(pos))
            pos[0] += 2
            AteDotsPos.append(tuple(pos))
            
        if dir_ == "Right":
            pos[1] -= i
            AteDotsPos.append(tuple(pos))
            pos[0] -= 1
            AteDotsPos.append(tuple(pos))
            pos[0] += 2
            AteDotsPos.append(tuple(pos))
            
        if dir_ == "Up":
            pos[1] += i
            AteDotsPos.append(tuple(pos))
            pos[0] -= 1 
            AteDotsPos.append(tuple(pos))
            pos[0] += 2
            AteDotsPos.append(tuple(pos))

        if dir_ == "Down":
            pos[1] -= i
            AteDotsPos.append(tuple(pos))
            pos[0] -= 1
            AteDotsPos.append(tuple(pos))
            pos[0] += 2
            AteDotsPos.append(tuple(pos))

In [200]:
def PacMove(current,pos,_map):            
    pos = list(pos)
    flag = False
    global AteDotsPos
    global PacDir 
    global Locations
    global Dirs
    
    for i in range (1,5):
        tmp = math.sqrt((Locations[i][0] - pos[0])**2 + (Locations[i][1] - pos[1])**2)
        if tmp <= 14:
            print("Game Over ...")
            return None,None,None

    flag = Access(pos,PacDir)
    dir_ = PacDir 
    if not flag:
        paths = Dirs.copy()
        if PacDir == "Left" or PacDir =="Right":  
            del paths[paths.index("Right")]     
            del paths[paths.index("Left")]  
            
        if PacDir == "Up" or PacDir =="Down":
            del paths[paths.index("Up")]     
            del paths[paths.index("Down")]
            
        PacDir = random.sample(paths,1)[0]
        flag = Access(pos,PacDir)
        if not flag:
            del paths[paths.index(PacDir)]
            PacDir = paths[0]  
            dir_ = PacDir          
        _map = PosWriter(pos,_map,label_imgs["Pacman Closed"])  
        return _map,pos,"Pacman Closed"

    
    new_pos = GetNewPos(pos,dir_) 
    if "Open" in current :
        current = "Pacman Half {0}".format(dir_)
        
    elif not(IsNotAte(new_pos,dir_,6)):
        if "Open" in current:
            current = "Pacman Half {0}".format(dir_)
        else :
            current = "Pacman Closed"

    elif "Open" in current and not(IsNotAte(new_pos,dir_,3)):
        current = "Pacman Half {0}".format(dir_)
        
    elif "Closed" in current and IsNotAte(new_pos,dir_,3):
        current = "Pacman Half {0}".format(dir_)
        
    elif "Half" in current and IsNotAte(new_pos,dir_,3):
        current = "Pacman Open {0}".format(dir_)
     
    EatPos(new_pos,PacDir,current) 
    AteDotsPos.append(tuple(new_pos)) 

    
    _map = PosWriter(new_pos,_map,label_imgs[current])  
    return _map,new_pos,current
        
 

In [201]:
def gen_demo(game_img, item_imgs):
    global FrameID 
    global States
    global Locations
    global PowerLocations
    FrameID += 1
    elements = GhostsNames.copy()
    elements.insert(0,"Pacman ")
    tmp = item_imgs["Maze_Empty"].copy()
    for _ in AteDotsPos:
        tmp[_[0]:_[0]+2,_[1]:_[1]+2,:] = [0,0,0]
        if _ in PowerLocations :
            index = PowerLocations.index(_)
            index = PowerLocations[(index//64)*64]
            tmp[index[0]:index[0]+8,index[1]:index[1]+8,:] = [0,0,0]
            for i in range(9):
                for j in range(9):
                    if tuple([index[0]+i,index[1]+j]) :
                        AteDotsPos.append(tuple([index[0]+i,index[1]+j]))
                
    for index in range(len(elements)):
        if elements[index] in States[index] :
            current = States[index]
        else: 
            current = elements[index] + States[index]

        if FrameID%3 == 0:
            if "1" in current:
                current = current.replace("Move1","Move2")
            else:
                current = current.replace("Move2","Move1")
                
        if "Pac" in elements[index]:
            tmp,pos,current = PacMove(current,Locations[index],tmp.copy())
            if current is None:
                return
        else:          
            tmp,pos,current = GhostMove(current,Locations[index],tmp.copy())
        Locations[index] = pos
        States[index] = current


    cv2.imwrite("./outputs/P04/{0}-map.png".format(FrameID),tmp)
    return tmp
            
            
            
            

while True:
    init = init_game(label_imgs["Maze"],label_imgs) 
    current_image = init
    IMAGES = []    
    for d in range(150):
        current_image = gen_demo(current_image,label_imgs)
        if current_image is None :
            break 
        IMAGES.append(current_image)
    print(len(IMAGES))
    if len(IMAGES) >= 70:
        break
    



150


In [202]:
h,w = label_imgs["Maze"].shape[0:2]
out = cv2.VideoWriter('./outputs/P04/00-Video Output.mp4',cv2.VideoWriter_fourcc(*'MP4V'), 8, (w,h))

for i in range(len(IMAGES)):
    out.write(IMAGES[i])
out.release()

In [221]:
finded_dots = cv2.matchTemplate(label_imgs["Maze"],label_imgs["Dot"], cv2.TM_CCORR_NORMED)
map_ = label_imgs["Maze"].copy()
w, h = label_imgs["Dot"].shape[1],label_imgs["Dot"].shape[0]
loc = np.where( finded_dots >= .99999)
for pt in zip(*loc[::-1]):
    cv2.rectangle(map_, pt, (pt[0] + w, pt[1] + h), (255,255,255), 2)
cv2.imwrite("./outputs/P04/00-Dot-Detection.png",map_)


True

In [222]:
finded_dots = cv2.matchTemplate(label_imgs["Maze"],label_imgs["Wall"], cv2.TM_CCORR_NORMED)
map_ = label_imgs["Maze"].copy()
w, h = label_imgs["Dot"].shape[1],label_imgs["Wall"].shape[0]
loc = np.where( finded_dots >= .99999)
for pt in zip(*loc[::-1]):
    cv2.rectangle(map_, pt, (pt[0] + w, pt[1] + h), (255,255,255), 2)
cv2.imwrite("./outputs/P04/00-Wall-Detection.png",map_)

True

In [223]:
finded_dots = cv2.matchTemplate(label_imgs["Maze"],label_imgs["Power"], cv2.TM_CCORR_NORMED)
map_ = label_imgs["Maze"].copy()
w, h = label_imgs["Dot"].shape[1],label_imgs["Power"].shape[0]
loc = np.where( finded_dots >= .99999)
for pt in zip(*loc[::-1]):
    cv2.rectangle(map_, pt, (pt[0] + w, pt[1] + h), (255,255,255), 2)
cv2.imwrite("./outputs/P04/00-Power-Detection.png",map_)

True

In [224]:
finded_dots = cv2.matchTemplate(label_imgs["Maze"],label_imgs["Power Pellet"], cv2.TM_CCORR_NORMED)
map_ = label_imgs["Maze"].copy()
w, h = label_imgs["Dot"].shape[1],label_imgs["Power Pellet"].shape[0]
loc = np.where( finded_dots >= .99999)
for pt in zip(*loc[::-1]):
    cv2.rectangle(map_, pt, (pt[0] + w, pt[1] + h), (255,255,255), 2)
cv2.imwrite("./outputs/P04/00-Power Pellet-Detection.png",map_)

True